# Week 5: LASSO (Coordinate Descent)

## Import GraphLab and Sales Data

In [3]:
import graphlab as gl

In [4]:
sales = gl.SFrame('kc_house_data.gl/')
# 'Floors' array was defined as type 'string'
# Convert to 'int' before using
sales['floors'] = sales['floors'].astype(int)

##Import Functions from Previous Notebook

In [9]:
import numpy as np

In [10]:
from regression import get_numpy_data, predict_output

## Normalize Features Practice

In [12]:
X = np.array([[3.0, 5.0, 8.0], [4.0, 12.0, 15.0]])

In [14]:
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


In [15]:
norms = np.linalg.norm(X, axis=0)
print norms

[  5.  13.  17.]


In [16]:
print X / norms

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Define a small function to return `normalized features`, i.e. the above line, and `norms`. 

In [17]:
def normalize_features(feature_matrix, verbose=False):
    X = np.array(feature_matrix)
    
    norms = np.linalg.norm(X, axis=0)
    normalized_features = X / norms
    
    if verbose is True:
        print('Normalize Features')
        print('feature matrix: \n%s\n' % feature_matrix)
        print('normalized features: \n%s\n' % normalized_features)
        print('norms: \n%s\n' % norms)
    
    return(normalized_features, norms)

In [18]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]), verbose=True)

Normalize Features
feature matrix: 
[[  3.   6.   9.]
 [  4.   8.  12.]]

normalized features: 
[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]

norms: 
[  5.  10.  15.]



## Implement Coordinate Descent with Normalized Features

## Effect of L1 Penalty

In [19]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output, verbose=True)

Get Numpy Data
data sframe: 
+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900.0 |   3.0    |    1.0    |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    1180.0   |   5650   |   1    |     0      |  0   |     3     |   7   |    1180    |
+-------------+----------+--------+------------+------+-----------+-------+------------+
+---------------+----------+--------------+---------+-------------+
| sqft_basement | yr_built | yr_renovated | zipcode |     lat     |
+----

In [20]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix, verbose=True)

Normalize Features
feature matrix: 
[[  1.00000000e+00   1.18000000e+03   3.00000000e+00]
 [  1.00000000e+00   2.57000000e+03   3.00000000e+00]
 [  1.00000000e+00   7.70000000e+02   2.00000000e+00]
 ..., 
 [  1.00000000e+00   1.02000000e+03   2.00000000e+00]
 [  1.00000000e+00   1.60000000e+03   3.00000000e+00]
 [  1.00000000e+00   1.02000000e+03   2.00000000e+00]]

normalized features: 
[[ 0.00680209  0.00353021  0.00583571]
 [ 0.00680209  0.00768869  0.00583571]
 [ 0.00680209  0.00230361  0.00389048]
 ..., 
 [ 0.00680209  0.00305154  0.00389048]
 [ 0.00680209  0.00478673  0.00583571]
 [ 0.00680209  0.00305154  0.00389048]]

norms: 
[  1.47013605e+02   3.34257264e+05   5.14075870e+02]



In [21]:
weights = np.array([1.0, 4.0, 1.0])

In [22]:
prediction = predict_output(simple_feature_matrix, weights, verbose=True)

Get Numpy Data
feature matrix 
[[ 0.00680209  0.00353021  0.00583571]
 [ 0.00680209  0.00768869  0.00583571]
 [ 0.00680209  0.00230361  0.00389048]
 ..., 
 [ 0.00680209  0.00305154  0.00389048]
 [ 0.00680209  0.00478673  0.00583571]
 [ 0.00680209  0.00305154  0.00389048]]

weights 
[ 1.  4.  1.]

predictions 
[ 0.02675867  0.04339256  0.01990703 ...,  0.02289873  0.03178473
  0.02289873]



In [23]:
rho = []

for i in range(len(weights)):
    feature_i = simple_feature_matrix[:, i]
    rho_i = (feature_i * (output - prediction + weights[i] * feature_i)).sum()
    
    rho.append(rho_i)

print('rho: \n%s\n' % rho)

rho: 
[79400300.034929156, 87939470.772991076, 80966698.675965652]



<a id="q1"></a>
### Quiz Question 1:
Recall that, whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?

<a id="q2"></a>
### Quiz Question 2:
What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

<a id="single"></a>
## Single Coordinate Descent Step

In [24]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty, show_input=False, show_output=False):
    # compute prediction
    norm_feature_matrix, norms = normalize_features(feature_matrix)
    prediction = predict_output(norm_feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature_i = norm_feature_matrix[:, i]
    rho_i  = (feature_i * (output - prediction + weights[i] * feature_i)).sum()
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = rho_i 
    elif rho_i < -l1_penalty / 2.0:
        new_weight_i = rho_i + l1_penalty / 2
    elif rho_i > l1_penalty / 2.0:
        new_weight_i = rho_i - l1_penalty / 2
    else:
        new_weight_i = 0.0
    
    if show_input is True:
        print('Lasso Coordinate Descent Step')
        print('i: \n%i\n' % i)
        print('feature matrix: \n%s\n' % str(feature_matrix))
        print('output: \n%s\n' % str(output))
        print('weights: \n%s\n' % weights)
        print('l1 penalty: \n%0.2f\n' % l1_penalty)
        
    if show_output is True:
        print('new weight i: \n%0.2f\n' % new_weight_i)
    
    return new_weight_i

In [25]:
# should print 0.425558846691
import math

test_i= 1
test_feature_matrix = np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                           [2./math.sqrt(13),3./math.sqrt(10)]])
test_output = np.array([1., 1.])
test_weights = np.array([1., 4.])
test_l1_penalty = 0.1

lasso_coordinate_descent_step(test_i, test_feature_matrix, test_output,
                              test_weights, test_l1_penalty, 
                              show_input=False, show_output=True)

new weight i: 
0.43



0.42555884669102573

<a id="cyclical"></a>
## Cyclical Coordinate Descent

In [26]:
from copy import copy

def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance, show_input=False, verbose=False, show_output=False):
    if verbose is True:
        print("Lasso Cyclical Coordinate Descent")
    
    # Create a shallow copy to keep from changing initial_weights
    weights = copy(initial_weights)
    
    # Set max iterations to ensure loop finishes
    max_iterations = 10000
    for n in range(max_iterations):
        # again will be set to True on every iteration of inner loop until
        # all diffs are less than tolerance
        again = False
        
        for i in range(len(weights)):
            # Save a copy before changing weight[i] so that diff can be taken
            old_weight_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            
            # Take diff between new and old weights
            diff = weights[i] - old_weight_i
            
            if diff > tolerance:
                again = True
            
        # Break if all diffs were less than tolerance
        if again is False:
            break
    
    # return None if loop completes without finding a minimum
    if again is True:
        return None
        
    if show_input is True:
        print("Lasso Cyclical Coordinate Descent")
        print('feature matrix: \n%s\n' % str(feature_matrix))
        print('output: \n%s\n' % str(output))
        print('initial weights: \n%s\n' % weights)
        print('l1 penalty: \n%0.2f\n' % l1_penalty)
        
    if show_output is True:
        print('weights: \n%s\n' % weights)
    
    return weights   

In [27]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance, 
                                            show_output=True)

weights: 
[ 21624999.22590417  63157245.99915871         0.        ]



<a id="q3"></a>
### Quiz Question 3:
What is the RSS of the learned model on the normalized dataset?

<a id="q4"></a>
### Quiz Question 4:
Which features had weight zero at convergence?